# Week 2.2 Representing Documents as Numbers
## NLP for the Creative Industries 
### Louis McCallum*
#### 2020
##### *who, after pushing it to its limits, has found out there are 5 levels of subheading in Mark-up
###### not 6, which the same as 5
####### Or 7, then it just turns into text and hashtags

Hopefully, you have followed the Git tutorial and have managed to update your repo and pull in this new notebook! 

If we want to find take a document or set of documents, and use them with machine learning techniques, or other mathematical operation to uncover some new information abou them, we can't just use the text and characters. We need a new representation, and that need to be numerical. This week we will be taking our first steps to representing collections of documents as numbers. 

- Tokenisation 
- Bags of Words 
- n-Grams
- TF/IDF
- Distances between documents 

### Building a Vocabulary 

The first step to getting a new, better representation of a text document is splitting it up into its consistuent parts. We call these **tokens** and deciding _what a token is_ in important choice. 

### Tokenisation 
We have already done some basic tokenisation in Week 1 using the `str.split()` method. Here we took a long string (multiple words) and split it into separate words (or "tokens") based on spaces. 

Does this seem sensible? Does this capture every thing that we would consider a separate word in the document? 

What about `isn't`? Is this one token (`isn't`), or two tokens (`is` and `not`)? Or `taxi cab`? Is that two tokens, do we care that `taxi` and `cab` are both used? Do we need this as a separate concept from `Uber`, `limo`, `Hackney Carriage` or `car`?. If we take it as one, do we miss out on other combinations like `black cab` or `taxi driver`? What about punctuation?

Just using `str.split()` works reasonably well on the sentence below. However, there are some issue with punctuation as ideally, the brackets and the exclaimation mark would also be separate tokens. 

After we have split our sentence into tokens, we can then create a **vocabulary**, which contains every unique token in the sentence.

In [ ]:
import numpy as np
#String split
sentence = "I like to think (it has to be!) of a cybernetic ecology where we are free of our labors and joined back to nature"
tokenised = sentence.split()
print(tokenised)
#Get the unique tokens (removes duplicates)
vocab = np.unique(tokenised)
print(vocab)

### Vectors and Matrices

A quick point on terminology before we move forwards. The arrays we have seen up until this point have mainly been 1  dimensional, that is all the items in the list are just single objects like numbers or strings. But, it is possible to have lists in multiple dimensions and for the time being we will just move to 2. 

It can help to think of a 1D array as a queue, or a shopping list. There are only two directions (backwards and forwards) and you only need **1 index** to access an item in it. You can think of 2D array is a grid, so more like a chess board. You can move in 4 directions (forwads, backwards, left and right) and you need **2 indexes** to access any item.

Technically, in a 2D array, each item of the outer array is also another 1D array.

Taking from mathematics, these 1D lists are often called **vectors** and 2D arrays are called **matrices** 

In [ ]:
#a is a matrix
a = np.array([[1,2],[3,4]])
print("a matrix")
print(a)
#Get the first row (a vector)
print("a vector", a[0])
#Get a specific item [row, col]
print(a[1,0])

### For Loops

Below we use a **for loop** to do this. This is something we use when we want to do a repeated action for a given number of times. The code below shows the standard structure. It tells us to take every item in `array` **in order**, and store it in `i`. The code underneath dictates what repeated actions we do with i each time and **must** be indented with a tab, otherwise Python will complain. This can be a single line code, or multiple lines. Every line that is indented will be included in the loop and executed each time.

`for i in array:
    do_something_with_i 
#end of loop
`
    
`for i in array:
     do_something_with_i
     do_something_else_with_i
     do_another_thing
#end of loop
`  

In [ ]:
#For loop prints out every item in turn
a = [1,2,3,4,5,10,15]
for i in a:
    print(i)
print("end of for loop 1")

In [ ]:
#Use range to get a sequence of numbers from 0->length of array
indexes = range(len(a))
for i in indexes:
    print(i,":",a[i])
print("end of for loop 2")


In [ ]:
#You can have multiple lines of code in the loop
#They are all indented 
for i in a:
    print(i)
    print(i + 2)
    print("this happens every time")
print("end of for loop 3, this only happens once")

### One Hot Encoding

Now that we have derived full vocabulary (if not exactly perfect yet) for the sentence, we can represent is a set of numbers. We take each token in the vocabulary and assign it an index, then for every token in the sentence, we create a vector that is all 0s, apart from a single 1 at the index taken from the vocabulary. 

In [ ]:
#Split into tokens based on spaces
tokenised = sentence.split()
#Get the unique tokens
vocab = np.unique(tokenised)
print(vocab)
#Make a matrix of zeros using the lengths of the separated sentence and vocab
one_hot = np.zeros((len(tokenised), len(vocab)))
#Go through the separated sentence and set the appropriate item to 1
for i in range(len(tokenised)):
    #Get the word
    word = tokenised[i]
    #find the index of the word in the vocab
    match = np.where(vocab == word)
    #Set it to 1 (hot)
    one_hot[i, match] = 1
print(one_hot)

This **one-hot encoding** doesn't lose any information, but its a lot of numbers for a small amount of information. This being said, it is also super **sparse**, which just means there are lots of 0s, and there are actually lots of techinques for really efficiently storing sparse data. 

We've successfully represented our sentence as a maxtrix of numbers, which we can use with various mathematical techniques moving forwards. 

### Bag of Words

Using **one-hot encoding**, we have a **vector** the length of the vocabulary for every word.

This can quickly get out of hand with longer documents and bigger vocabularies.

One improvement we can make to this representation is to simply count up every occurence of each word in the vocabulary for each document, and then store this count for each word in the vocabulary. 

This means we only have one **word frequency vector** for each document, rather than a one-hot encoding vector for each word. If we had multiple documents (or sentences), we could make a **word frequency vector** for each one and store them as a **matrix** (2D array).

The dictionary that we get out of the Counter object is an efficient storage method, as absent words are just ignored.

Even though this a big compression of the data, this approach actually ends up not losing much of the meaning of the document. 

In [ ]:
#Use a Counter to create a Bag of Words (word-frequency vector)
from collections import Counter
bow = Counter(tokenised)
bow

### Looking at Books
Here we have a novel from [Project Gutenberg](https://gutenberg.org/ebooks/). Its about hacking is and copyright free. Lets see what we can find out about it.

What can we find out about each chapter by counting the amount of times a word appears in each? Are any similar to each other? How can we adjust our vocabulary to help us out? First we're going to look at the book as a whole.

We're going to use a number of techniques to try and tweak our vocabulary so that it contains the most information for when we start using this bag of words in tasks like topic modelling or classification. 

This means we want to count things that have the same meaning as the same token, but we also don't want to throw away any information that might help our processing. 

In [ ]:
import re
fs = open('hacking.txt', 'r') 
book = fs.read()

In [ ]:
#tokens is a 1D array
tokens = book.split()
#Get the unique tokens (our vocabulary)
vocab = np.unique(tokens)
print("total words:",len(tokens), "unique words:", len(vocab))
#Create a Bag of Words using a Counter
Counter(tokens).most_common(200)

In [ ]:
#Find all words that have a . or , in them. 
ctr = 0;
for word in vocab:
    if len(re.findall(r'[\.,;!?]$', word)) > 0:
        if word[:-1] in vocab:
            ctr = ctr + 1
print(ctr," duplicate words ending in [.,;!?] out of ", len(vocab))

### Removing punctuation duplicates
We can see lots of tokens have trailing punctutation, and a good few of these will also exist in without the punctuation. This duplication is bad for us!

We would want these to be the same token so we can use a **regex** to replace it. The regex below splits on whitespace (represented by `\s`) or punctuation that appears at least once (using this plus notation `+`). We immediately see the size of the vocabulary drops by about 25%, showing there was loads of duplication in our bag of words. 

In [ ]:
#Use a regex to split based on space AND punctuation
tokens = re.split(r'[-\s.,;!?]+', book)
vocab = np.unique(tokens)
print("unique words", vocab.shape)
#Counter(tokens).most_common(50)

### Stop words
We can see that the commonly occurring words don't tell us much about this specific book. Traditionally, in NLP it has been useful to remove words that occur commonly. They don't tell us very much about each document because they are contained in almost all the documents. These are known as **stop words**. 

In contemporary NLP we often don't actually remove stop words because we have the computing power to deal with the extra vocab size and any information we throw away can effect performance, especially in deep learning, and especially when we start to look at context of sequences of words. 

Here we see a list from the sklearn library (each library has its own list of stop words). 

We'll just see how removing stop words from our bag effects what we can see. Although our vocabulary size is basically the same (so we're not saving much in effiency), the list most common words are much more informative and tells us more about the specific book we're looking at. 

In [ ]:
#Install library if not already installed
!pip install sklearn

In [ ]:
from sklearn.feature_extraction import stop_words
stop_tokens = []
for t in tokens:
    if not t in stop_words.ENGLISH_STOP_WORDS:
        stop_tokens = stop_tokens + [t]
stop_vocab = np.unique(stop_tokens)
print("unique words", stop_vocab.shape)
Counter(stop_tokens).most_common(50)

### Stemming
Stemming attempt to remove suffixes from words that contain the same base. This reduces variation and can help when we reduce the documents into a more distilled form (like a bag of words). 

- hacking, hackers, hacked, hacks
- computer, computing, computers

Depending on our task, its probably the case that we only really care about knowing if **any** of these words appear, not whether they each appear individually. For example, I might be doing a search for paragraphs about hacking and it may be that I would miss out on key documents otherwise if I only searched for one of the words. 

Stemming can be quite a challenging task however. If we want to combine pluralisations, for example, we can't just remove the "s" from the end of all nouns, what about 

- grass (not a plural)
- mice, octopi (plural, no s)
- geniuses (plural, es)

We're going to use the built in stemmer in the NLTK library. This reduces our vocabulary in the hacking book dramatically!

In [ ]:
#Install the nltk library 
!pip install nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
word_list = ['feet', 'foot', 'foots', 'footing']
for word in word_list:
    print(word, "->", stemmer.stem(word))
#Doesn't always work
word_list = ['organise','organises','organised','organisation',"organs","organ","organic"]
for word in word_list:
    print(word, "->", stemmer.stem(word))

In [ ]:
#Looking at our hacking book
stem_tokens = []
for t in stop_tokens:
    stem_tokens = stem_tokens + [stemmer.stem(t)]
stem_vocab = np.unique(stem_tokens)
print("unique words", stem_vocab.shape)
Counter(stem_tokens).most_common(50)

### Lemmatisation 
Lemmatisation is a technique similar to stemming, apart from it attempts to find similar meanings, as opposed to just similar roots. Like with all these _normalisation_ techniques, reducing your vocabulary will reduce precision but may make your model bettter at generalising and more efficient.

For example, lemmatisation would be able to separate **dogged** and **dog**, which have quite different meanings but would get combined by a stemmer. 

Below we use the WordNetLemmatizer from the NLTK library. 

In [ ]:
import nltk
nltk.download("wordnet")
from nltk.stem import WordNetLemmatizer
lem = WordNetLemmatizer()

In [ ]:
print(lem.lemmatize("dogged"))
print(lem.lemmatize("dog"))
print(stemmer.stem("dogged"))
print(stemmer.stem("dog"))
print(lem.lemmatize("better", pos ="a"))
print(lem.lemmatize("better", pos = "v"))
print(lem.lemmatize("betterment", pos = "n"))

In [ ]:
#Looking at our hacking book
lem_tokens = []
for t in stop_tokens:
    lem_tokens = lem_tokens + [lem.lemmatize(t)]
lem_vocab = np.unique(lem_tokens)
print("unique words", lem_vocab.shape)
Counter(lem_tokens).most_common(50)

### Capitalisation
Whilst it may be tempting to just lower case every token with the belief that words all have the same meaning regardless of case. However, it may actually be that if something is in ALL CAPS it conveys some meaning. Or if a word is at the start of sentence, that has importance. 

For example 

- John liked to help
- John screamed HELP HELP HELP

Or if one book contained lots of capitalised nouns (like cities and countries), it might tell you it was about Geography.

Some libraries actually account for this by lower casing everything, then having a token which indicates a start of capitilising as well as one that signifies the end of capitalising. This allows the best of both worlds. Of course, this only works if your model or vocabulary is able to take sequence and context into account. Like for example....

In [ ]:
lower_tokens = []
for word in lem_tokens:
    lower_tokens = lower_tokens + [word.lower()]
lower_vocab = np.unique(lower_tokens)
print("unique words", lower_vocab.shape)
bow = Counter(lower_tokens)
bow.most_common(50)

### n-Grams
When we do Bag of Words, what we gain in effiency and generalisation, we lose in context. We can gain lots by including combinations of words as tokens, and these are known as **n-grams**. For example, if our book was a tale about an butcher / computer programmer / hacky-sack apologist, we could could see the term `hacking` in a bunch of contexts. If we don't look at the subsequent token, we lose this information.

 - hacking meat
 - hacking computers
 - hacking sack
 
On a more realistic note, I see that `phone` and `network` appear almost exactly the same amount of times, and I'd wager this is because they follow one another and using multiword tokens will cpature this connection. Also consider **not**. A newspaper article that said someone was `not guilty` and `not under arrest` and `not a bad chap`, could be summarised with `guilty`, `arrest` and `bad` with just single word tokens.
 
A 2-gram makes a token out of very pair of subsequent words, and a 3-gram out of every 3 etc.

What we see when we apply this to our book is that there is a massive explosion in the number of tokens. Which makes sense, as we're counting seqeuences and its likely that words will appear in more than sequence, and that exact matching sequences will appear less times that single words. 

But we also see some new things, for example, `australian` by itself didnt make the top 50, but when combined with to make `australian hacker`, it becomes much more common in respect to the whole document. This quickly lets us know perhaps of all the hackers in the book, an australian one is the most prominent. Or if there's only one hacker, they're most prominent feature is their australian-ness. 

And although '`computer` is very regular in the single word list, we get more context with 2-grams such as `computer undergroud` and `computer crime`.

Some of thes combinations will be super rare and its common to drop any n-grams that don't appear often from your vocabulary. When your feature vector (the size of the bag of words) gets bigger than the number of documents you have, you start to get problems. 

In [ ]:
n = 2
ngram_tokens = []
#Loop through all the tokens
for i in range(len(lower_tokens)-n):
    word = lower_tokens[i]
    #Stich together with subsequent tokens into one string
    for j in range(n - 1):
        word = word + " " + lower_tokens[i + 1 + j]
    #Add to list
    ngram_tokens = ngram_tokens + [word]
ngram_vocab = np.unique(ngram_tokens)
print("unique words", ngram_vocab.shape)
Counter(ngram_tokens).most_common(50)

### TF/IDF

Up until this point, we've seen how counting words, and looking at the most frequent can gives us some insight into a single document. If we want to start comparing documents with more certainty, or getting smarter about our representations, we can try and get a set of numbers for each documents that not only represents **word frequency**, but also **word importance**. 

What we will end up with is a measurement called **TF/IDF** or **T**erm **F**requency x **I**nverse **D**ocument **F**requency. 

### TF

**TF** stands for **term frequency** and we've been using it a lot already in our Bags of Words. By itself, it tells us how many times a particular term appears in a document. Can we do better?

Consider our book and some of its most common words

- computer 
- hacking
- security 
- police
- network

These words seem to represent key topics of the book quite well. However, what about **mother**? This appears 113  times across the book, out of a vocabulary of  approx. 13,000 words. Compare this to a WhatsApp conversation that me and my sister had about our family Christmas that has the word **mother** 5 times in with a vocabulary of about 50 words. When we compare just **term frequency**, it seems like the hacking book is far, far more (~20 times) about mothers than this text message chain. But thats not really the case. 

We use **normalised term frequency** to account for this, where the length of the document is used alongside the count to adjust for this.


In [ ]:
#Divide term frequency by total number of unique words (vocab size)
book_tf = bow["mother"] / len(lower_vocab)
text_msg_tf = 5 / 50
#Much bigger normalised term frequency for text msgs
print(book_tf, text_msg_tf)

### IDF

**IDF** stands for **I**nverse **D**ocument **F**requency and it tells us how important a word is in a particular document in comparison to the rest of the corpus. Up until this point we've been considering the book as one big document, but now we're going to take each chapter on its own, to see if we can see if we can highlight differences between them.

We can see below that most chapters have the terms **computer** and **hacker** featuring pretty heavily. The **IDF** is the ratio of all documents in comparison to how many documents the term appears in. It tells us how surprising is it that this word appeared here, given what we know about all the documents. 

First, we use a **regex** to split it into chapters, as there is a recognisable formatting to this. This means our corpus is the whole novel, with each chapter considered a new document and we store the whole thing as a 1D array. Each item in the array is a string containing a chapter's worth of text.

### Getting the TF/IDF Vector for each document

In [ ]:
#First get the Bag of Words for each chapter 
bow_chapters = []
#Split on Chapters
book = re.split(r'\s\s\s\s\s\sChapter+', book)
for chapter in book:
    #Split on spaces and punctuation
    tokens = re.split(r'[-\s.,;!?]+', chapter)
    processed = []
    for t in tokens:
        #Lemmatise and make lowercase
        t = lem.lemmatize(t.lower())
        if not t in stop_words.ENGLISH_STOP_WORDS:
            processed = processed + [t]
    bow = Counter(processed)
    print("****new chapter****")
    print(bow.most_common(10))
    bow_chapters = bow_chapters + [bow]   

In [ ]:
#Get all the vocabulary across all chapters
vocab = []
for chapter in bow_chapters:
    vocab = vocab + list(chapter.keys())
vocab = np.unique(vocab)

tf_chapters = []
num_docs = len(book)
vocab_size = len(vocab)
docs_containing_word = np.zeros(len(vocab))
#Go through each chapter
for i in range(num_docs):
    tf = []
    #Go through each term in the vocab
    for j, term in enumerate(vocab):
        count = 0;
        #If the term is present in the chapter, collect data
        if bow_chapters[i][term]:
            #Save the normalised TF
            count = bow_chapters[i][term] / vocab_size
            #Note that this word appeared in this doc (document frequency)
            docs_containing_word[j] = docs_containing_word[j] + 1
        #Add to array of terms
        tf = tf + [count]
    #Add to array of chapters
    tf_chapters = tf_chapters + [tf]

tf_chapters = np.array(tf_chapters)
print(tf_chapters.shape)
tfidf = np.zeros(tf_chapters.shape)
#For every chapter
for i in range(num_docs):
    #For every term
    for j in range(vocab_size):
        #Calculate TF/IDF
        tfidf[i, j] = tf_chapters[i, j] * np.log((num_docs / docs_containing_word[j]))


### Examining the highest TF/IDF values

Now we which words are important to each chapter. Interestingly we've lost all of the words like `computer` and `hacking`, because they're surprising or indicative of that chapter, given the whole corpus. These words are the words that tell us the most about each chapter.

It seems likes names (of people and of viruses?) are important distinctions between chapters. 

We also did lemmatisation instead of stemming and often have the same word, and its possesive version in a chapter (`anthrax` and `anthrax's`). Maybe stemming would be better?

In [ ]:
#Store in a dataframe and sort
import pandas as pd
#Use the vocab as the column names
data = pd.DataFrame(tfidf, columns = vocab)
for i in range(len(tfidf)):
    print("chapter", i)
    print(data.iloc[i].sort_values(ascending = False).head(10))

### Maths With Word Vectors
So what we have now is a **vector** for each document (in our case, each document is a chapter). This vector represents something about the text in that chapter based on the frequencies that words occur, and how that relates to the corpus as a whole. 

We can use these vectors calculate how similar two documents by calculating the distance between them. Our vectors are currently >10,000. This means this is the _dimensionality_ of our vector. We can actually use similar maths that we would use to work out the distance between 2 points in 2 dimensional space. And its much easier to visualise how that works!

Two methods often used are **Manhattan distance** and **Euclidean** distance, but we tend to use something else for TF/IDF vectors.

### Cosine Distance

What we actually want to use is something called the **cosine distance**, which essentially tells how much the two vectors are pointing in the same direction. The results go from -1 to 1, where 1 is exactly the same, 0 is nothing in common and -1 is **anti-similar**. However, this never happens for TFIDF vectors, because word counts can never be negative!

### Topic Modelling 

What we already see is that we can begin to group documents together by how similar they are. Next week Rebecca will teach you some more advanced methods for taking this idea further. 

Interestingly, the first chapter seems to be the most different from the rest, and I think that isn't a Chapter per se, but the preface. Also, consecutive chapters tend to be the most simliar to each other. 

In [ ]:
book[0]

In [ ]:
#Import the cosine similarity method from sklearn
from sklearn.metrics.pairwise import cosine_similarity as cosine
result = cosine(tfidf)
#Put the result in a dataframe and 
df = pd.DataFrame(result)
#Show with heatmap style gradients
df.style.background_gradient(cmap='Greens')

### Round up

So here we've seen how we can take a bunch of text documents, and represent them as a collection of numerical vectors. This representation means we can start to use basic maths to compare documents, and we'll see in future weeks how this representation is the building block of tonnes of other more complex processing, including machine learning models. 

When getting this new representation, depending on our task, we want something that 

- Is as few dimensions as possible **BUT** maintains as much information as possible. Its always a balance and will depend on your goals, modelling approach and dataset.
- Correctly captures the important points of document 
- Highlights the differences between documents